In [1]:
import pandas as pd
from collections import Counter
import time
# from data_test import *
# from mushroom import *
# from retail_transaction_dataset import *
# from fruithut.fruithut import *

In [2]:
# sample
data = {
    'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
    'Items': [['a', 'c', 'd'],
              ['a', 'b', 'd'],
              ['b', 'c', 'd', 'e'],
              ['a', 'd'],
              ['c', 'd', 'e'],
              ['a', 'b', 'c', 'd', 'e']]
}
df = pd.DataFrame(data)

# data = {
#     'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
#     'Items': [['apple', 'cherry', 'durian'],
#               ['apple', 'banana', 'durian'],
#               ['banana', 'cherry', 'durian', 'elderberry'],
#               ['apple', 'durian'],
#               ['cherry', 'durian', 'elderberry'],
#               ['apple', 'banana', 'cherry', 'durian', 'elderberry']]
# }

# df = pd.DataFrame(data)
df['Item_Length'] = df['Items'].apply(lambda items: len(items))
len_df = len(df)
df

,Tid,Items,Item_Length
0,T1,"[a, c, d]",3
1,T2,"[a, b, d]",3
2,T3,"[b, c, d, e]",4
3,T4,"[a, d]",2
4,T5,"[c, d, e]",3
5,T6,"[a, b, c, d, e]",5


In [3]:
unique_items = sorted(df['Items'].explode().unique()) # get unique item => save to list
unique_items

['a', 'b', 'c', 'd', 'e']

In [4]:
length_transaction = df["Items"].apply(len)
if length_transaction.nunique() == 1:
    hastheSameLengh = True
else:
    hastheSameLengh = False

hastheSameLengh

False

In [5]:
length_transaction = length_transaction.to_list()
length_transaction

[3, 3, 4, 2, 3, 5]

In [6]:
# calculate stset: {'a': [T1, T2, T4, T6]} - list Tid containing unique item
def cal_stset(df):
    stset = {} 
    for item in unique_items:
        tid_list = df[df['Items'].apply(lambda items: item in items)]['Tid'].tolist() # create column Items with items in unique item
        tid_lengths = [len(df[df['Tid'] == tid]['Items'].iloc[0]) for tid in tid_list if item in df[df['Tid'] == tid]['Items'].iloc[0]]
        stset[item] = {"StSet": tid_list, "Length_transaction": tid_lengths} # add value with item_key

    df_stset = pd.DataFrame.from_dict(stset, orient = 'index').reset_index()
    df_stset.columns = ["Items", "StSet", "Length_transaction"]
    df_stset['Items'] = df_stset['Items'].apply(lambda x: [x]) 
    return df_stset

In [7]:
# calculate support - count number of Tid containing unique item
def cal_support(df_stset):
    df_support = pd.DataFrame(columns=['Items', 'Support'])
    df_support['Items'] = df_stset['Items']
    df_support['Support'] = df_stset['StSet'].apply(len)
    return df_support

In [8]:
df_stset = cal_stset(df)
df_stset

,Items,StSet,Length_transaction
0,[a],"[T1, T2, T4, T6]","[3, 3, 2, 5]"
1,[b],"[T2, T3, T6]","[3, 4, 5]"
2,[c],"[T1, T3, T5, T6]","[3, 4, 3, 5]"
3,[d],"[T1, T2, T3, T4, T5, T6]","[3, 3, 4, 2, 3, 5]"
4,[e],"[T3, T5, T6]","[4, 3, 5]"


In [9]:
df_support = cal_support(df_stset)
df_support

,Items,Support
0,[a],4
1,[b],3
2,[c],4
3,[d],6
4,[e],3
